# Use case examples
This notebook demonstrates different use-cases for QC Lab

## Running a default simulation

In [1]:
from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state["wf_db"] = np.array([1.0, 0.0], dtype=complex)
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim)

100%|██████████| 10001/10001 [00:01<00:00, 6009.43it/s]


## Changing the model ingredients

In [2]:
from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np
from qclab.ingredients import init_classical_wigner_harmonic

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state["wf_db"] = np.array([1.0, 0.0], dtype=complex)
print(sim.model.ingredients[5])
# change the init_classical ingredient from boltzmann to wigner.
sim.model.ingredients.insert(0,('init_classical', init_classical_wigner_harmonic))
print(sim.model.ingredients)
print(sim.model.ingredients[5])
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim)

('init_classical', <function init_classical_boltzmann_harmonic at 0x1112b7b80>)
[('init_classical', <function init_classical_wigner_harmonic at 0x1112b7c10>), ('h_q', <function h_q_two_level at 0x1112b7820>), ('h_qc', <function h_qc_diagonal_linear at 0x1112b7940>), ('h_c', <function h_c_harmonic at 0x111256ee0>), ('dh_qc_dzc', <function dh_qc_dzc_diagonal_linear at 0x1112b79d0>), ('dh_c_dzc', <function dh_c_dzc_harmonic at 0x1112b7700>), ('init_classical', <function init_classical_boltzmann_harmonic at 0x1112b7b80>), ('hop', <function hop_harmonic at 0x1112b7a60>), ('_init_h_q', <function SpinBoson._init_h_q at 0x1112bb550>), ('_init_h_qc', <function SpinBoson._init_h_qc at 0x1112bb5e0>), ('_init_model', <function SpinBoson._init_model at 0x1112bb700>), ('_init_h_c', <function SpinBoson._init_h_c at 0x1112bb670>)]
('dh_c_dzc', <function dh_c_dzc_harmonic at 0x1112b7700>)


100%|██████████| 10001/10001 [00:01<00:00, 6094.79it/s]


## Changing algorithm tasks

There is a known incompatibility between this use case (changing a task within a Jupyter notebook environment) and the use of the multiprocessing driver in MacOS. This incompatibility is not present, however, in a python script.

In [ ]:
from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state["wf_db"] = np.array([1.0, 0.0], dtype=complex)
print(sim.algorithm.collect_recipe)
def collect_response(sim, state, parameters):
    # we can just calculate something simple like the response function
    if sim.t_ind == 0:
        state["wf_db_0"] = np.copy(state["wf_db"])
    state["response"] = np.sum(np.conj(state["wf_db_0"]) * state["wf_db"], axis=-1)
    state["output_dict"]['response'] = state["response"]
    return state, parameters
sim.algorithm.collect_recipe.append(collect_response)
print(sim.algorithm.collect_recipe)
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim, num_tasks=4)

## User supplied seeds



In [4]:
np.random.seed(1234)

my_seeds = np.unique(np.random.randint(0,10000, size=100))

from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state["wf_db"] = np.array([1.0, 0.0], dtype=complex)
data_serial = serial_driver(sim, seeds=my_seeds)

assert np.all(data_serial.data_dict['seed'] == my_seeds)
print('validated serial_driver with seeds')

data_parallel = parallel_driver_multiprocessing(sim, seeds=my_seeds)

assert np.all(data_parallel.data_dict['seed'] == my_seeds)
print('validated parallel_driver_multiprocessing with seeds')

for key, val in data_serial.data_dict.items():
    if key in data_parallel.data_dict:
        assert np.all(val == data_parallel.data_dict[key]), f"Mismatch in {key} between serial and parallel drivers"
print('validated data consistency between serial and parallel drivers')


100%|██████████| 10001/10001 [00:00<00:00, 17775.48it/s]

validated serial_driver with seeds



 93%|█████████▎| 9285/10001 [00:01<00:00, 11038.13it/s]

validated parallel_driver_multiprocessing with seeds
validated data consistency between serial and parallel drivers


100%|██████████| 10001/10001 [00:01<00:00, 5791.22it/s]


## User supplied data object

In [5]:
## Serial
from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state["wf_db"] = np.array([1.0, 0.0], dtype=complex)
data_1 = serial_driver(sim)
data_2 = serial_driver(sim, data=data_1)

sim_settings = {"num_trajs":200,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state["wf_db"] = np.array([1.0, 0.0], dtype=complex)
data_3 = serial_driver(sim)

for key, val in data_3.data_dict.items():
    if key in data_2.data_dict:
        assert np.all(val == data_2.data_dict[key]), f"Mismatch in {key}"
print('validated data consistency between supplied data and new simulation')

100%|██████████| 10001/10001 [00:00<00:00, 17498.36it/s]

validated data consistency between supplied data and new simulation


In [ ]:
## Parallel
from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state["wf_db"] = np.array([1.0, 0.0], dtype=complex)
data_1 = parallel_driver_multiprocessing(sim)
data_2 = parallel_driver_multiprocessing(sim, data=data_1)
print(len(data_2.data_dict['seed']))
sim_settings = {"num_trajs":200,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state["wf_db"] = np.array([1.0, 0.0], dtype=complex)
data_3 = parallel_driver_multiprocessing(sim)

for key, val in data_3.data_dict.items():
    if key in data_2.data_dict:
        print(f"Validating {key}")
        assert np.all(val == data_2.data_dict[key]), f"Mismatch in {key}"
print('validated data consistency between supplied data and new simulation')

100%|██████████| 10001/10001 [00:02<00:00, 4657.57it/s]


200


100%|██████████| 10001/10001 [00:03<00:00, 2906.74it/s]


Validating seed
Validating norm_factor
Validating t
Validating dm_db
Validating classical_energy
Validating quantum_energy
validated data consistency between supplied data and new simulation
